# Preprocess Indo

In [ ]:
from gensim.models import Doc2Vec
import re
import os
import pandas as pd
import numpy as np
import multiprocessing
import ipython_genutils
import pickle
import progressbar
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import WordPunctTokenizer
from keras.layers.embeddings import Embedding
from sklearn.metrics import classification_report
from sklearn import model_selection
from sklearn import utils
from sklearn.utils import shuffle
from gensim.models.word2vec import Word2Vec
from keras.preprocessing.text import Tokenizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk

### load file

In [ ]:
df = pd.read_csv('lib/dataset ahok',header=None)
df=df.rename(columns={0:'target',1:'text'})
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
df.info()
df.head()

In [ ]:
df.text[8]

### hapus emoji

In [ ]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [ ]:
widgets = [progressbar.Percentage(),progressbar.Bar()," Processed : ",progressbar.Counter(),"  ",progressbar.ETA()]
bar = progressbar.ProgressBar(widgets=widgets, max_value=len(df.index))
bar.start()

t=[]
for index,row in bar(df.iterrows()):
    t.append(remove_emoji(row['text']))
df.text=t

### cleaning

In [ ]:
widgets = [progressbar.Percentage(),progressbar.Bar()," Processed : ",progressbar.Counter(),"  ",progressbar.ETA()]
bar = progressbar.ProgressBar(widgets=widgets, max_value=len(df.index))
bar.start()

tok=WordPunctTokenizer()
t=[]
for index,row in bar(df.iterrows()):
    x=row["text"]
    # hapus rt
    cl = re.sub(r'^(RT|rt)( @\w*)?[: ]', '', x)
    #cl = re.sub(r'\s*RT\s*@[^:]*:.*', '', x)
    #cl = re.sub(r'\s*rt\s*@[^:]*:.*', '', cl)
    # hapus mention
    cl = re.sub(r'@[A-Za-z0-9]([^:\s]+)+', '', cl)
    # hapus emoticon
    cl = re.sub(r'(\:\w+\:|\<[\/\\]?3|[\(\)\\\D|\*\$][\-\^]?[\:\;\=]|[\:\;\=B8][\-\^]?[3DOPp\@\$\*\\\)\(\/\|])(?=\s|[\!\.\?]|$)', '', cl)
    # hapus link
    cl = re.sub(r'http?://[A-Za-z0-9./]+', '', cl)
    cl = re.sub(r'https?://[A-Za-z0-9./]+', '', cl)  
    # hapus hashtag
    cl = re.sub(r'#[A-Za-z0-9\-\.\_]+(?:\s|$)', '', cl)
    cl = re.sub(r'(?:\s|^)#[A-Za-z0-9\-\.\_]+(?:\s|$)', '', cl)
    # kata ulang
    cl = re.sub(r'\w*\d\w*', '', cl)
    cl = re.sub(r'\b(\w+)(\1\b)+', r'\1', cl)
    # hapus simbol
    cl = re.sub(r'[^a-zA-Z]', ' ', cl)
    # lower
    cl=cl.lower()
    # format teks 
    cl=tok.tokenize(cl)
    cl=(" ".join(cl))
    t.append(cl)
    #print(cl)
df.text=t

In [ ]:
df.text[8]

### hapus row yg terdapat data kosong

In [ ]:
ex=''
df=df[df.text != ex]
df.reset_index(drop=True,inplace=True)
df.info()

### hapus stopwords

In [ ]:
widgets = [progressbar.Percentage(),progressbar.Bar()," Processed : ",progressbar.Counter(),"  ",progressbar.ETA()]
bar = progressbar.ProgressBar(widgets=widgets, max_value=len(df.index))
bar.start()

# stopwords sastrawi
factory = StopWordRemoverFactory()

# tambah stopwords ke dict sastrawi
more_stopwords=[line.strip() for line in open('lib/more_stopwords.txt')]
factory.get_stop_words()+more_stopwords
stopwords = factory.create_stop_word_remover()

t=[]
for index, row in bar(df.iterrows()):
    row["text"] = stopwords.remove(row["text"])
    t.append(row['text'])
    #print(row["text"])
df.text=t

In [ ]:
df.text[8]

### stemming

In [ ]:
# spacy
from spacy.lang.id import Indonesian
import spacy
nlp = Indonesian() 
nlp = spacy.blank('id') 

widgets = [progressbar.Percentage(),progressbar.Bar()," Processed : ",progressbar.Counter(),"  ",progressbar.ETA()]
bar = progressbar.ProgressBar(widgets=widgets, max_value=len(df.index))
bar.start()


lemma=[]
for index, row in bar(df.iterrows()):
    temp=[]
    doc = nlp(row['text'])
    for x in doc:
        temp.append(x.lemma_)
    lemma.append(" ".join(temp))
df.text=lemma  

In [ ]:
df.text[8]

In [ ]:
# sastrawi

widgets = [progressbar.Percentage(),progressbar.Bar()," Processed : ",progressbar.Counter(),"  ",progressbar.ETA()]
bar = progressbar.ProgressBar(widgets=widgets, max_value=len(df.index))
bar.start()

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

t=[]
for index, row in bar(df.iterrows()):
    t.append(stemmer.stem(row["text"]))
    #print(row["text"])
#df.text=t


In [ ]:
t[8]

In [ ]:
df.text[8]

In [ ]:
df.text=t

### save file

In [ ]:
df.to_csv('lib/dataset', encoding='utf-8')